![](https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/se05.png)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLDiego/uom_fse_dl_workshop/blob/main/solutions/SE05_SL_Transfer_Learning.ipynb)

## Workshop Overview
***
In this workshop, we explore transfer learning techniques for medical image segmentation. Starting from a custom dataset pipeline, we build a U-Net architecture from scratch and then leverage a pre-trained EfficientNet encoder to demonstrate the power of transfer learning.

**Prerequisites**: CNNs (SE04), Neural networks (SE02), PyTorch fundamentals (SE01)

**Learning Objectives**:
- Understand transfer learning concepts and when to apply them
- Create custom PyTorch datasets for image segmentation tasks
- Compute dataset statistics for normalization
- Apply Albumentations for synchronized image and mask augmentation
- Build a U-Net architecture for medical image segmentation
- Implement and apply the Dice loss for binary segmentation
- Apply transfer learning using a pre-trained EfficientNet encoder


In [ ]:
import sys
import subprocess

if "google.colab" in sys.modules:
    print("Running in Google Colab: downloading utils...")
    subprocess.run([
        "wget",
        "-q",
        "--show-progress",
        "https://raw.githubusercontent.com/CLDiego/uom_fse_dl_workshop/main/colab_utils.txt",
        "-O",
        "colab_utils.txt",
    ], check=True)
    subprocess.run([
        "wget",
        "-q",
        "--show-progress",
        "-x",
        "-nH",
        "--cut-dirs=3",
        "-i",
        "colab_utils.txt",
    ], check=True)
else:
    print("Running locally: skipping Colab utils download.")

In [ ]:
from pathlib import Path
import sys

repo_path = Path.cwd()
if str(repo_path) not in sys.path:
    sys.path.append(str(repo_path))

import utils
from torchvision import transforms
import random
import torch
import numpy as np
from PIL import Image

# Set random seeds for reproducibility
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU device:", torch.cuda.get_device_name(0))
else:
    print("No GPU available. Please ensure you've enabled GPU in Runtime > Change runtime type")
print(f"Using device: {device}")


# 1. Introduction to Transfer Learning
***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/write.svg" width="20"/> **Definition**: Transfer learning is a machine learning technique where a model developed for one task is reused as a starting point for a model on a second task. It's particularly effective for deep learning models, as it allows us to leverage pre-trained models' knowledge rather than starting from scratch.

In previous sessions, we learned how to build and train neural networks from scratch. However, training large deep learning models requires:

1. **Massive datasets** (often millions of examples)
2. **Extensive computational resources** (often multiple GPUs)
3. **Long training times** (days to weeks)

Transfer learning addresses these challenges by letting us capitalise on existing models that have already been trained on large datasets.

<div align="center">
  <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/transfer.png" width="50%">
</div>

## 1.1 Transfer Learning Analogy
***

Transfer learning is inspired by human learning. Consider how we learn:

| Human Learning | Machine Learning Parallel |
|----------------|--------------------------|
| A child learns to recognize basic shapes before identifying letters | A model learns edge detection before specific object recognition |
| A musician who knows piano can learn guitar faster than a novice | A model trained on one image dataset can adapt quickly to a similar task |
| Language skills transfer across related languages (e.g., Spanish to Italian) | NLP models pre-trained on one language can be fine-tuned for another |
| Medical students learn general anatomy before specializing | Medical imaging models trained on general X-rays can be fine-tuned for specific conditions |
| Engineers apply fundamental principles across different projects | Engineering models transfer physical principles across different applications |

This mirrors how neural networks learn hierarchical features. Early layers learn general patterns that are often applicable across domains, while later layers learn task-specific features.


## 1.2 When to Use Transfer Learning
***

Transfer learning is particularly useful in the following scenarios:

| Scenario | Example | Benefit |
|----------|---------|---------|
| **Limited training data** | Medical imaging with few samples | Pre-trained features compensate for data scarcity |
| **Similar domains** | From natural images to satellite imagery | Underlying features (edges, textures) transfer well |
| **Time constraints** | Rapid prototyping needs | Accelerates model development cycle |
| **Hardware limitations** | Training with limited GPU access | Reduces computational requirements |
| **Preventing overfitting** | Small dataset applications | Regularization effect from pre-trained weights |
***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> **Key Insight**: The effectiveness of transfer learning depends on the similarity between the source and target domains. The more similar they are, the more beneficial transfer learning becomes.


# 2. Case Study: Image Segmentation for Medical Imaging
***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/write.svg" width="20"/> **Image Segmentation**: The process of partitioning an image into multiple segments or regions, often used in medical imaging to identify and delineate structures within images (e.g., tumors, organs). It is a crucial step in many computer vision tasks, including object detection and recognition.

For this session, we are going to be using the [**ISIC 2016 Skin Lesion Segmentation Challenge**](https://challenge.isic-archive.com/landing/2016/) dataset. This dataset contains dermoscopic images of skin lesions, along with their corresponding segmentation masks. The goal is to train a model to accurately segment the lesions from the background.

In [ ]:
data_path = Path(Path.cwd(), 'datasets')
dataset_path = utils.data.download_dataset('skin lesions',
                                           dest_path=data_path,
                                           extract=True,
                                           remove_compressed=False)

mask_path = utils.data.download_dataset('skin lesions masks',
                                   dest_path=data_path,
                                   extract=True,
                                   remove_compressed=False)

test_path = utils.data.download_dataset('skin lesions test',
                                   dest_path=data_path,
                                   extract=True,
                                   remove_compressed=False)

test_mask_path = utils.data.download_dataset('skin lesions test masks',
                                   dest_path=data_path,
                                   extract=True,
                                   remove_compressed=False)

## 2.1 Challenges in Medical Image Segmentation
***

Medical image segmentation presents unique challenges compared to natural image segmentation:

| Challenge | Description | Impact |
|-----------|-------------|--------|
| **Limited Data** | Medical datasets are typically smaller | Transfer learning becomes crucial |
| **Class Imbalance** | Regions of interest often occupy a small portion of the image | Requires specialized loss functions |
| **Ambiguous Boundaries** | Boundaries between tissues can be gradual or unclear | Makes precise segmentation difficult |
| **Inter-observer Variability** | Different experts may segment the same image differently | Ground truth is not always definitive |
| **High Stakes** | Errors can have serious consequences in medical applications | Demands higher accuracy and reliability |

# 3. Preparing the Dataset
***
Segmentation tasks require both the input images and their corresponding masks. The masks are binary images where the pixels belonging to the object of interest (e.g., a tumor) are marked as 1 (or white), while the background is marked as 0 (or black). Thus, we need to load both the images and their masks for training.

## 3.1 Custom Dataset Creation
***

In order for us to efficiently load the images and masks, we are going to create a custom dataset class. This class will inherit from the `torch.utils.data.Dataset` class and will handle loading the images and masks from the specified directories.

The PyTorch `Dataset` class is an abstract class representing a dataset. Custom datasets should inherit from this class and override the following methods:

| Method | Purpose | Implementation Requirements |
|--------|---------|------------------------------|
| `__init__` | Initialize the dataset | Define directories, transformations, and data loading parameters |
| `__len__` | Return dataset size | Return the total number of samples |
| `__getitem__` | Access a specific sample | Load and transform a sample with a given index |

For image segmentation, our dataset needs to handle both input images and their corresponding segmentation masks:

***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Snippet 1**: Create a custom dataset class for loading

``` python
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, **kwargs):
        """
        Initializes the dataset, loading the images and masks from the specified directories.
        """
    
    def __len__(self):
        """
        Returns the number of images in the dataset.
        """
    
    def __getitem__(self, idx):
        """
        Defines how to get a single item (image and mask) from the dataset.
        
```

<img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Exercise 1**: Implement a custom `ISICDataset` class inheriting from `torch.utils.data.Dataset`, with `__init__`, `__len__`, and `__getitem__` methods that load images and masks using PIL and apply optional transforms.


In [ ]:

# Exercise 1: Creating a Custom Dataset
# In this exercise, you will implement a custom PyTorch Dataset class for loading the ISIC skin lesion images and masks.

from torch.utils.data import Dataset, DataLoader

class ISICDataset(Dataset):
    def __init__(self, image_dir: Path | str, mask_dir: Path | str, img_transform=None, mask_transform=None):
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.img_transform = img_transform
        self.mask_transform = mask_transform
        ###################
        # TODO: COMPLETE THE CODE BELOW
        # Get sorted list of image files (*.jpg) from image_dir, and mask files (*.png) from mask_dir
        self.images = ___
        self.masks = ___

    def __len__(self):
        return ___  # Return the number of images in the dataset

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = self.image_dir / img_name.name
        # Mask filename: stem of img_name + '_segmentation.png'
        mask_name = ___ + '_segmentation.png'
        mask_path = self.mask_dir / mask_name

        # Open image as RGB and mask as grayscale ('L')
        image = Image.open(img_path).convert(___)
        mask = Image.open(mask_path).convert(___)

        if self.img_transform:
            image = ___(image)
        if self.mask_transform:
            mask = ___(mask)
        return image, mask

resize_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

ds = ISICDataset(
    image_dir=dataset_path,
    mask_dir=mask_path,
    img_transform=resize_transform,
    mask_transform=resize_transform
)


## 3.2 Compute the Mean and Standard Deviation of the Dataset
***

Computing dataset statistics is a critical step in preparing data for deep learning models. We need to normalize our images to help the model converge faster and perform better. By normalizing with the dataset's mean and standard deviation, we ensure that the input values have similar scales and distributions.

The normalisation process follows this formula for each channel:

$$x_{normalized} = \frac{x - \mu}{\sigma}$$

Where:
- $x$ is the original pixel value
- $\mu$ is the mean of all pixels in the channel across the dataset
- $\sigma$ is the standard deviation of all pixels in the channel across the dataset

First, we need to load the dataset and then compute the mean and standard deviation across all images.

<img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Exercise 2**: Create a `DataLoader` for the dataset and iterate through it to compute the per-channel mean and standard deviation of all images.


In [ ]:

# Exercise 2: Calculating Mean and Std of Images
# In this exercise, you will create a DataLoader and compute the per-channel mean and standard deviation of the dataset.

from tqdm import tqdm

###################
# TODO: COMPLETE THE CODE BELOW
# Create a DataLoader with batch size 16 and shuffle=False
dl = DataLoader(___, batch_size=16, shuffle=___)

n_channels = 3
n_pixels = 0
channel_sum = torch.zeros(n_channels)
channel_squared_sum = torch.zeros(n_channels)

for images, _ in tqdm(dl, desc="Calculating mean and std"):
    # Reshape to (batch_size, channels, height*width)
    images = images.view(images.size(0), n_channels, ___)
    # Update pixel count (batch_size * pixels_per_channel)
    n_pixels += ___

    # Update sums (sum over batch and spatial dimensions)
    channel_sum += images.sum(dim=___)
    channel_squared_sum += (images ** 2).sum(dim=___)

# Calculate mean as sum divided by total pixel count
mean = ___
# Calculate std using: sqrt((sum_of_squares / count) - mean^2)
std = torch.sqrt(___)

print("Mean:", mean)
print("Std:", std)


## 3.3 Data Augmentation
***
We are going to use the albumentations library for data augmentation. This library outperforms the torchvision library in terms of speed and flexibility. It provides the same transformations as torchvision and it is also compatible with PyTorch.

### 3.3.1 Choosing Appropriate Augmentations for Medical Images

| Augmentation Type | Purpose | Medical Imaging Considerations |
|-------------------|---------|-------------------------------|
| **Geometric Transforms** | Rotate, flip, resize | Should preserve diagnostic features |
| **Color Adjustments** | Brightness, contrast, saturation | Use carefully to maintain diagnostic appearance |
| **Noise Addition** | Add random noise | Models will be more robust to image noise |
| **Elastic Deformations** | Simulate tissue deformation | Especially useful for soft tissue imaging |
| **Cropping** | Focus on different regions | Ensures focus on different areas of lesion |

### 3.3.2 Sync vs. Async Augmentation

For segmentation tasks, we need to ensure that the same transformations are applied to both the image and its corresponding mask. This is called synchronized (sync) augmentation, as opposed to asynchronous (async) augmentation where different transformations are applied to inputs and targets.

| Type | Description | Use Case |
|------|-------------|----------|
| **Sync Augmentation** | Apply identical spatial transforms to image and mask | Required for segmentation tasks |
| **Async Augmentation** | Apply different transforms | Typically used for classification only |
***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> **Important**: When augmenting for segmentation tasks, always ensure that geometric transformations (flips, rotations, etc.) are applied identically to both the image and its mask to maintain pixel-to-pixel correspondence.
***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Snippet 2**: Using albumentations for data augmentation

``` python
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=mean, std=std, p=1.0),
    ToTensorV2()
])
```

<img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Exercise 3**: Build an Albumentations training pipeline (with resize to 64×64, flips, rotation, colour jitter, normalisation, and `ToTensorV2`) and a validation pipeline (with resize and normalisation only).


In [ ]:

# Exercise 3: Data Augmentation with Albumentations
# In this exercise, you will create training and validation augmentation pipelines using the Albumentations library.

import albumentations as A
from albumentations.pytorch import ToTensorV2

###################
# TODO: COMPLETE THE CODE BELOW
# Training pipeline: Resize to 64×64, HorizontalFlip (p=0.5), VerticalFlip (p=0.5),
# Rotate (limit=10, p=0.5), ColorJitter, Normalize, ToTensorV2
train_img_ts = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=___),
    A.VerticalFlip(p=___),
    A.Rotate(limit=___, p=0.5),
    A.ColorJitter(brightness=___, contrast=___, saturation=___, hue=___, p=0.5),
    A.Normalize(mean=mean.tolist(), std=std.tolist(), p=1.0),
    ___()  # Convert to PyTorch tensor
])

# Validation pipeline: Resize to 64×64, Normalize, ToTensorV2 (no augmentation)
valid_img_ts = A.Compose([
    A.Resize(64, 64),
    A.Normalize(mean=mean.tolist(), std=std.tolist(), p=1.0),
    ___()
])


### 3.3.4 Modifying the Dataset Class to use Albumentations
***
Since normal PyTorch transforms do not support the synchronized augmentation, we need to modify our dataset class to use albumentations. We will also add the normalization step in the `__getitem__` method.

We are going to create a new class called that inherits from our `ISICDataset` class. This will allow us to override the `__getitem__` method and apply the albumentations transformations.

<img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Exercise 4**: Create `ISICDatasetAlbumentations` inheriting from `ISICDataset`, override `__getitem__` to convert images and masks to NumPy arrays, apply Albumentations transforms synchronously, and ensure masks are binary (0 or 1).


In [ ]:

# Exercise 4: Implementing the Albumentations Dataset Class
# In this exercise, you will extend ISICDataset to apply synchronized Albumentations transforms to both images and masks.

class ISICDatasetAlbumentations(ISICDataset):
    def __init__(self, image_dir: Path | str, mask_dir: Path | str, transform=None):
        super().__init__(image_dir, mask_dir, transform, None)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = self.image_dir / img_name.name
        mask_name = img_name.stem + '_segmentation.png'
        mask_path = self.mask_dir / mask_name

        ###################
        # TODO: COMPLETE THE CODE BELOW
        # Open image as RGB and mask as grayscale
        image = Image.open(img_path).convert(___)
        # Convert to numpy array for albumentations
        image = np.array(___)
        mask = Image.open(mask_path).convert(___)
        mask = np.array(___)

        # Normalize mask to 0-1 range if max > 1
        if mask.max() > 1:
            mask = ___  # divide by 255.0

        if self.img_transform:
            # Apply albumentations transform to both image and mask jointly
            aug = self.img_transform(image=___, mask=___)
            image = aug['image']
            mask = aug['mask']

            # Ensure mask is binary (0 or 1)
            if isinstance(mask, torch.Tensor):
                mask = (mask > ___).float()

        return image, mask


# 3.4 Splitting the Dataset into Train, and Validation Sets
***
Unfortunately, PyTorch `Dataset` class does not have a built-in method for splitting datasets. However, we can use the `torch.utils.data.random_split` function to split our dataset into training and validation sets. Then, we can create separate `DataLoader` instances for each split.

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Snippet 3**: Splitting the dataset into training and validation sets

``` python
from torch.utils.data import random_split

train_ds, val_ds = random_split(dataset, [train_size, val_size])
```

<img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Exercise 5**: Split the full `ISICDatasetAlbumentations` dataset 80/20 into training and validation sets, then wrap each split in a `DataLoader` with batch size 16.


In [ ]:

# Exercise 5: Splitting the Dataset into Train and Validation Sets
# In this exercise, you will split the dataset 80/20 and create DataLoaders for each split.

from torch.utils.data import random_split

###################
# TODO: COMPLETE THE CODE BELOW
full_ds = ISICDatasetAlbumentations(
    image_dir=dataset_path,
    mask_dir=mask_path,
    transform=___  # training transforms
)

# Calculate 80/20 split sizes
train_size = int(___ * len(full_ds))
valid_size = len(full_ds) - ___

# Split the dataset into training and validation sets
train_ds, valid_ds = random_split(full_ds, [___, ___])

# Create DataLoaders (batch size 16; shuffle training, don't shuffle validation)
train_dl = DataLoader(train_ds, batch_size=___, shuffle=___)
valid_dl = DataLoader(valid_ds, batch_size=___, shuffle=___)


In [ ]:
# Show a batch of images and masks
utils.plotting.show_binary_segmentation_batch(train_dl,
                                              n_images=10,
                                              mean=mean,
                                              std=std)

# 4. Baseline Model: U-Net Architecture
***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/write.svg" width="20" /> **U-Net**: A convolutional neural network architecture designed for biomedical image segmentation. It consists of a contracting path (encoder) and an expansive path (decoder), allowing it to capture both context and localization information.

The U-Net architecture is widely used in medical image segmentation tasks due to its ability to learn both local and global features. The architecture is shown below:

<div align="center">
  <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/unet.png" width="100%">
</div>


The U-Net architecture consists of two main parts: the encoder and the decoder, connected by skip connections. Each component plays a specific role:

| Component | Description | Purpose |
|-----------|-------------|---------|
| **Encoder (Contracting Path)** | Series of convolutional and pooling layers | Captures context and semantic information |
| **Decoder (Expanding Path)** | Series of upsampling and convolutional layers | Enables precise localization |
| **Skip Connections** | Connect encoder layers to decoder layers | Preserve spatial information lost during downsampling |
| **Bottleneck** | Deepest layer connecting encoder and decoder | Captures the most complex features |

## 4.1 Transposed Convolution
***

For this architecture we are going to use a special type of convolutional layer that upsamples the input feature maps. This layer is called a transposed convolutional layer (also known as a deconvolutional layer). It is used to increase the spatial dimensions of the input feature maps, allowing the model to learn more complex features.

| Parameter | Description | Effect on Output |
|-----------|-------------|-----------------|
| **Kernel Size** | Size of the filter | Determines area of influence |
| **Stride** | Step size of the filter | Controls amount of upsampling |
| **Padding** | Zero-padding added to input | Affects output size |
| **Output Padding** | Additional padding for output | Fine-tunes output dimensions |

The formula for calculating the output size of a transposed convolutional layer is:

$$\text{Output Size} = (\text{Input Size} - 1) \times \text{Stride} - 2 \times \text{Padding} + \text{Kernel Size} + \text{Output Padding}$$

## 4.2 U-Net Implementation in PyTorch
***
We are going to implement the U-Net architecture using PyTorch. The implementation will consist of the following components:

| Component | Description |
|----------|-------------|
| `DoubleConv` | A block that consists of two convolutional layers followed by batch normalization and ReLU activation. |
| `Down` | A block that consists of a max pooling layer followed by a `DoubleConv` block. |
| `Up` | A block that consists of a transposed convolutional layer followed by a `DoubleConv` block. |
| `UNet` | The main U-Net architecture that consists of the encoder and decoder blocks. |

In [ ]:

# Exercise 6: Implementing the DoubleConv Block
# In this exercise, you will implement the foundational double-convolution block used in U-Net.

class DoubleConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        ###################
        # TODO: COMPLETE THE CODE BELOW
        # Create a sequential module with two conv layers, each followed by BatchNorm2d and ReLU
        # Use kernel_size=3, padding=1 for both conv layers
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=___, padding=___),
            torch.nn.BatchNorm2d(___),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(out_channels, out_channels, kernel_size=___, padding=___),
            torch.nn.BatchNorm2d(___),
            torch.nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return ___(x)


In [ ]:

# Exercise 7: Implementing the Down Block
# In this exercise, you will implement the encoder (downsampling) block with max pooling followed by DoubleConv.

class Down(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        ###################
        # TODO: COMPLETE THE CODE BELOW
        # Create a sequential module: MaxPool2d(2) followed by DoubleConv
        self.maxpool_conv = torch.nn.Sequential(
            torch.nn.MaxPool2d(___),
            DoubleConv(___, ___)
        )

    def forward(self, x):
        return ___(x)


In [ ]:

# Exercise 8: Implementing the Up Block
# In this exercise, you will implement the decoder (upsampling) block with transposed convolution and skip connections.

class Up(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        ###################
        # TODO: COMPLETE THE CODE BELOW
        # ConvTranspose2d: in_channels -> in_channels//2, kernel_size=2, stride=2
        self.up = torch.nn.ConvTranspose2d(___, ___, kernel_size=___, stride=___)
        # DoubleConv takes concatenated channels (in_channels full) -> out_channels
        self.conv = DoubleConv(___, ___)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # Calculate padding to match x2's spatial dimensions
        diffY = x2.size(2) - x1.size(2)
        diffX = x2.size(3) - x1.size(3)
        
        x1 = torch.nn.functional.pad(x1, [diffX // 2, diffX - diffX // 2,
                                          diffY // 2, diffY - diffY // 2])
        
        # Concatenate along the channel dimension, then apply conv
        x = torch.cat([___, ___], dim=1)
        return self.conv(___)


In [ ]:

# Exercise 9: Implementing the UNet Model
# In this exercise, you will assemble the full U-Net architecture using the DoubleConv, Down, and Up blocks.

class UNet(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super().__init__()
        
        ###################
        # TODO: COMPLETE THE CODE BELOW
        # Initial convolution block
        self.inc = DoubleConv(___, 64)
        
        # Encoder path (increasing channel depths)
        self.down1 = Down(64, ___)
        self.down2 = Down(___, 256)
        self.down3 = Down(256, ___)
        self.down4 = Down(___, 1024)
        
        # Decoder path (decreasing channel depths)
        self.up1 = Up(1024, ___)
        self.up2 = Up(___, 256)
        self.up3 = Up(256, ___)
        self.up4 = Up(___, 64)
        
        # Output layer: 64 channels -> out_channels, kernel_size=1
        self.outc = torch.nn.Conv2d(64, ___, kernel_size=1)

    def forward(self, x):
        # Encoder path
        x1 = self.inc(___)
        x2 = self.down1(___)
        x3 = self.down2(___)
        x4 = self.down3(___)
        x5 = self.down4(___)
        
        # Decoder path with skip connections
        x = self.up1(x5, ___)
        x = self.up2(x, ___)
        x = self.up3(x, ___)
        x = self.up4(x, ___)
        
        output = self.outc(___)
        return torch.nn.functional.sigmoid(___)


## 4.3 Segmentation Loss Functions
***
While we can use a simple loss function like binary cross-entropy for segmentation tasks since we are dealing with binary masks, it is often not sufficient. This is because the model may learn to predict the background class (0) more often than the foreground class (1), leading to poor performance on the actual segmentation task.

To address this, we are going to introduce a more sophisticated loss function called the **Dice Loss**. The Dice Loss is based on the Dice coefficient, which measures the overlap between two sets. It is defined as:

$$
\text{Dice} = \frac{2 |X \cap Y|}{|X| + |Y|}$$

Where:
- $X$ is the predicted segmentation mask
- $Y$ is the ground truth segmentation mask
- $|X|$ is the number of pixels in the predicted mask
- $|Y|$ is the number of pixels in the ground truth mask
- $|X \cap Y|$ is the number of pixels in the intersection of the predicted and ground truth masks

<div align="center">
  <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/dice.png" width="80%">
</div>


The Dice Loss is defined as:
$$
\text{Dice Loss} = 1 - \text{Dice}
$$

### 4.3.1 Comparison of Segmentation Loss Functions
***
| Loss Function | Formula | Advantages | Disadvantages |
|---------------|---------|------------|---------------|
| **Binary Cross-Entropy** | $-\sum(y\log(\hat{y})+(1-y)\log(1-\hat{y}))$ | Easy to implement, works well for balanced classes | Poor performance with class imbalance |
| **Dice Loss** | $1 - \frac{2\|X \cap Y\|}{\|X\| + \|Y\|}$ | Handles class imbalance well, directly optimizes overlap | May get stuck in local minima |
| **Focal Loss** | $-\alpha(1-\hat{y})^\gamma y\log(\hat{y})$ | Focuses on hard examples, addresses class imbalance | Requires tuning of hyperparameters |
| **IoU Loss** | $1 - \frac{\|X \cap Y\|}{\|X \cup Y\|}$ | Directly optimizes intersection over union | Can be unstable for small regions |
| **Combo Loss** | $\alpha\cdot BCE + (1-\alpha)\cdot Dice$ | Combines benefits of both BCE and Dice | Requires tuning of weighting parameter |

This loss function is particularly useful for imbalanced datasets, where the number of pixels in the foreground class is much smaller than the number of pixels in the background class. The Dice Loss penalizes the model more for misclassifying foreground pixels than background pixels, leading to better performance on the segmentation task.

<img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Exercise 10**: Implement the `DiceLoss` class inheriting from `nn.Module`, with a `smooth` parameter and a `forward` method that computes the Dice loss from flattened predictions and targets.


In [ ]:

# Exercise 10: Implementing the Dice Loss Function
# In this exercise, you will implement the Dice loss function for binary image segmentation.

class DiceLoss(torch.nn.Module):
    def __init__(self, smooth=1.0):
        super().__init__()
        ###################
        # TODO: COMPLETE THE CODE BELOW
        self.smooth = ___

    def forward(self, y_pred, y_true):
        # Flatten the prediction and target tensors to 1D
        y_pred = y_pred.view(___)
        # Convert target to float
        y_true = y_true.view(___).float()
        
        # Calculate intersection (element-wise multiplication, then sum)
        intersection = (y_pred * y_true).sum()
        # Calculate union (sum of each tensor)
        union = y_pred.sum() + y_true.sum()
        
        # Calculate Dice coefficient with smoothing: (2*intersection + smooth) / (union + smooth)
        dice = ___
        
        # Return loss clamped between 0 and 1
        return torch.clamp(1 - dice, 0.0, 1.0)


In [ ]:
# Initialize the model, criterion, and optimizer

model = UNet(in_channels=3, out_channels=1).to(device)
criterion = DiceLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 2

model_v1 = utils.ml.train_model(
    model=model,
    criterion=criterion,
    optimiser=optimiser,
    train_loader=train_dl,
    val_loader=valid_dl,
    num_epochs=num_epochs,
    early_stopping=True,
    patience=3,
    save_path=Path.cwd() / "my_models" / "se05_model_v1.pt",
    plot_loss=True,
)


In [ ]:
# Show a batch of images and masks
test_ds = ISICDatasetAlbumentations(
    image_dir=test_path,
    mask_dir=test_mask_path,
    transform=valid_img_ts
)
test_dl = DataLoader(test_ds, batch_size=16, shuffle=True)

utils.plotting.show_binary_segmentation_predictions(model_v1, test_dl, n_images=10, mean=mean, std=std)

# 5. Transfer Learning with Pre-trained Models
***
In essence, the process of transfer learning involves taking a model that has been trained on a large dataset (the source domain) and adapting it to a new, smaller dataset (the target domain). This is done by reusing the learned features from the source model and fine-tuning them for the target task. The most common approach is to use the pre-trained model as a feature extractor, where the lower layers of the model are frozen and using a classifier head that is trained on the new dataset.

The process can be visualized as follows:

<div align="center">
  <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/transfer_custom.png" width="60%">
</div>

## 5.1 Pre-trained Models for Computer Vision
***

Several pre-trained models are available for computer vision tasks. Each has its own architecture, number of parameters, and performance characteristics:

| Model | Parameters | Input Size | Year | Top-1 Accuracy (ImageNet) | Architecture Highlights |
|-------|------------|------------|------|---------------------------|-------------------------|
| **ResNet** | 11.7M - 60M | 224×224 | 2015 | 76.1% - 80.6% | Residual connections to combat vanishing gradients |
| **VGG** | 138M - 144M | 224×224 | 2014 | 71.3% - 75.6% | Simple architecture with small filters (3×3) |
| **Inception** | 6.8M - 54M | 299×299 | 2014 | 77.5% - 82.8% | Multi-scale processing with parallel paths |
| **DenseNet** | 8M - 44M | 224×224 | 2017 | 74.5% - 79.5% | Dense connections between layers for feature reuse |
| **EfficientNet** | 5.3M - 66M | 224×224 - 600×600 | 2019 | 78.8% - 85.7% | Balanced scaling of depth, width, and resolution |
| **MobileNet** | 4.2M - 6.9M | 224×224 | 2017 | 70.6% - 75.2% | Designed for mobile devices with depthwise separable convolutions |

In this section, we are going to use a pre-trained model for the segmentation task. We are going to use an EfficientNet model that has been pre-trained on the ImageNet dataset. The EfficientNet model is a state-of-the-art convolutional neural network architecture that achieves high accuracy with fewer parameters compared to other architectures.

<div align="center">
  <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/efficient.png" width="80%">
</div>

## 5.2 Transfer Learning Process
***

The typical transfer learning workflow consists of these steps:

| Step | Description | Technique |
|------|-------------|-----------|
| **1. Select Source Model** | Choose a pre-trained model relevant to your target task | Select models trained on large datasets like ImageNet |
| **2. Feature Extraction** | Use the pre-trained model as a fixed feature extractor | Freeze pre-trained layers, replace and retrain output layers |
| **3. Fine-tuning** | Carefully adapt pre-trained weights to the new task | Gradually unfreeze layers, train with lower learning rates |
| **4. Model Adaptation** | Modify architecture if needed for the target task | Add or remove layers as needed for the new domain |

## 5.3 Types of Transfer Learning
***

There are several approaches to implementing transfer learning:

| Approach | Description | Best Used When |
|----------|-------------|---------------|
| **Feature Extraction** | Freeze pre-trained network, replace and retrain classifier | Target task is similar but dataset is small |
| **Fine-Tuning** | Retrain some or all layers of pre-trained network | Target task has sufficient data but benefits from pre-training |
| **One-shot Learning** | Learn from just one or very few examples | Extreme data scarcity |
| **Domain Adaptation** | Adapt to new data distribution without labels | Source and target domains have distribution shift |
| **Multi-task Learning** | Train model on multiple related tasks simultaneously | Related tasks can benefit from shared representations |

## 5.4 EfficientNet as an Encoder
***

The EfficientNet model is going to act as the encoder part of the U-Net architecture. We are going to replace the encoder part of the U-Net architecture with the EfficientNet model. The decoder part of the U-Net architecture will remain the same.

<div align="center">
  <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/unet_efficient.png" width="80%">
</div>

### 5.4.1 Transfer Learning Process for Segmentation
***
| Step | Description | Implementation Detail |
|------|-------------|----------------------|
| **1. Extract Encoder** | Use pre-trained EfficientNet as encoder | Remove classification head |
| **2. Add Decoder** | Create U-Net style decoder | Transposed convolutions with skip connections |
| **3. Freeze Weights** | Prevent pre-trained encoder from changing | Set `requires_grad=False` on encoder layers |
| **4. Train Decoder** | Train only the decoder initially | Optimize only unfrozen parameters |
| **5. Fine-tune (Optional)** | Gradually unfreeze encoder layers | Use lower learning rate for pre-trained layers |
***
> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Snippet 4**: Loading the EfficientNet model and freezing layers

``` python
from torchvision import models as tvm

efficientnet = tvm.efficientnet_b0(weights='IMAGENET1K_V1')
efficientnet = efficientnet.features  # Extract the feature extractor part
efficientnet = torch.nn.Sequential(*list(efficientnet.children())[:-1])  # Remove the classification head

for param in efficientnet.parameters():
    param.requires_grad = False  # Freeze all layers
```

<img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> **Exercise 11**: Implement `UNetEfficient` using a pre-trained EfficientNet-B0 encoder with frozen weights, a center bottleneck, and a `SmartUp` decoder path with skip connections, producing binary segmentation outputs via sigmoid.


In [ ]:

# Exercise 11: Implementing U-Net with EfficientNet Encoder
# In this exercise, you will build a U-Net variant using a pre-trained EfficientNet-B0 as a frozen encoder.

import torchvision.models as tvm

class SmartUp(torch.nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super(SmartUp, self).__init__()
        ###################
        # TODO: COMPLETE THE CODE BELOW
        # ConvTranspose2d: in_channels -> skip_channels, kernel_size=2, stride=2
        self.up = torch.nn.ConvTranspose2d(___, ___, kernel_size=___, stride=___)
        # DoubleConv takes skip_channels*2 (concatenation) -> out_channels
        self.conv = DoubleConv(___ * 2, ___)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # Handle size mismatches with bilinear interpolation
        if x1.size()[2:] != x2.size()[2:]:
            x1 = torch.nn.functional.interpolate(x1, size=x2.size()[2:], mode='bilinear', align_corners=False)
        
        x = torch.cat([___, ___], dim=1)
        return self.conv(___)
    
class UNetEfficient(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNetEfficient, self).__init__()
        
        # Load pretrained EfficientNet-B0
        efficient_net = tvm.efficientnet_b0(weights=___)
        
        # Extract feature layers - using UNet encoder nomenclature
        # Hint: efficient_net.features is a list of blocks [0..8]
        self.inc = torch.nn.Sequential(*list(efficient_net.features)[:2])  # Initial block
        self.down1 = efficient_net.features[___ ]                           # MBConv block 2
        self.down2 = efficient_net.features[___ ]                           # MBConv block 3
        self.down3 = efficient_net.features[___ ]                           # MBConv block 4
        self.down4 = torch.nn.Sequential(                                   # MBConv blocks 5,6,7
            efficient_net.features[___],
            efficient_net.features[___],
            efficient_net.features[___]
        )
        
        # Freeze encoder layers to preserve pretrained weights
        encoders = [self.inc, self.down1, self.down2, self.down3, self.down4]
        for encoder in encoders:
            for param in encoder.parameters():
                param.requires_grad = ___  # Set to False to freeze
        
        # Output channel sizes of each encoder stage
        enc1_channels = 16
        enc2_channels = 24
        enc3_channels = 40
        enc4_channels = 80
        enc5_channels = 320
        
        # Center bottleneck
        self.center = DoubleConv(___, 512)
        
        # Decoder path with SmartUp blocks
        self.up1 = SmartUp(512, ___, enc4_channels)
        self.up2 = SmartUp(enc4_channels, ___, enc3_channels)
        self.up3 = SmartUp(enc3_channels, ___, enc2_channels)
        self.up4 = SmartUp(enc2_channels, ___, enc1_channels)
        
        # Final 1×1 output convolution
        self.outc = torch.nn.Conv2d(___, out_channels, kernel_size=1)
    
    def forward(self, x):
        input_size = x.size()[2:]
        
        # Encoder path
        x1 = self.inc(___)
        x2 = self.down1(___)
        x3 = self.down2(___)
        x4 = self.down3(___)
        x5 = self.down4(___)
        
        # Bottleneck
        x = self.center(___)
        
        # Decoder path with skip connections
        x = self.up1(x, ___)
        x = self.up2(x, ___)
        x = self.up3(x, ___)
        x = self.up4(x, ___)
        
        output = self.outc(___)
        
        # Resize to original input dimensions if needed
        if output.size()[2:] != input_size:
            output = torch.nn.functional.interpolate(output, size=input_size, mode='bilinear', align_corners=False)
        
        return torch.nn.functional.sigmoid(___)


In [ ]:
# Initialize the EfficientNet UNet model
model_efficient = UNetEfficient(in_channels=3, out_channels=1).to(device)
criterion_efficient = DiceLoss()
optimizer_efficient = torch.optim.Adam(filter(lambda p: p.requires_grad, model_efficient.parameters()), lr=1e-3)

num_epochs = 5

model_efficient = utils.ml.train_model(
    model=model_efficient,
    criterion=criterion_efficient,
    optimiser=optimizer_efficient,
    train_loader=train_dl,
    val_loader=valid_dl,
    num_epochs=num_epochs,
    early_stopping=True,
    patience=3,
    save_path= Path.cwd() / "my_models" / "se05_model_v2.pt",
    plot_loss=True,
)

In [ ]:
# Visualize predictions with EfficientNet UNet
utils.plotting.compare_binary_segmentation_models(
    model_v1, model_efficient, test_dl, n_images=10, mean=mean, std=std)

## 5.5 Advantages of Transfer Learning
***
Transfer learning is particularly effective in computer vision and natural language processing (NLP) tasks, where large pre-trained models are available. The key advantages of transfer learning include:

| Advantage | Description | Impact |
|-----------|-------------|--------|
| **Reduced Training Time** | Start with pre-learned features instead of random weights | Training can be 5-10x faster than from scratch |
| **Less Training Data** | Leverage knowledge from the source domain | Can work with hundreds vs. thousands of examples |
| **Better Performance** | Often achieves higher accuracy than training from scratch | Especially beneficial with limited target data |
| **Faster Convergence** | Models typically reach optimal performance in fewer epochs | Reduces computational costs of model development |
| **Lower Computational Cost** | Requires fewer resources for training | Makes deep learning accessible with limited hardware |
| **Knowledge Retention** | Preserves useful features learned from large datasets | Captures generalizable representations across domains |
